# Experiment with the tuner

In [5]:
%reload_ext autoreload
%autoreload 2

# from common.metrics.gradient import Gradient
import torch
from torch.utils.data import DataLoader
import torchinfo
import numpy as np
import pandas as pd
import pytorch_lightning as pl 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
from pylab import *
import ast
import os
import sys
from statistics import mean
import warnings

# import modules from ECON model
module_path = os.path.abspath(os.path.join('../tuner/')) # or the path to your source code
sys.path.insert(0, module_path)
from jet_tuner import JetTagger

module_path = os.path.abspath(os.path.join('../../workspace/models/jets/code/')) # or the path to your source code
sys.path.insert(0, module_path)
from jet_datamodule import JetDataModule

# import modules from common metrics
module_path = os.path.abspath(os.path.join('../../workspace/common/metrics/')) # or the path to your source code
sys.path.insert(0, module_path)
from CKA import CKA
from neural_efficiency import NeuralEfficiency


# import modules from common benchmarks
module_path = os.path.abspath(os.path.join('../../workspace/common/benchmarks/')) # or the path to your source code
sys.path.insert(0, module_path)
from noisy_dataset import NoisyDataset


DATA_PATH="/home/jovyan/checkpoint/"    
DATASET_DIR="../../data/JTAG"
DATASET_FILE=DATASET_DIR + "/processed_dataset.h5"

In [7]:
model_path = '../tuner/experiment_models/net_best.pkl'
ne_dataloader = JetDataModule(DATASET_FILE, 
                              data_dir=DATASET_DIR,
                              batch_size=1, 
                              num_workers=8).test_dataloader()

# load the model
model = JetTagger(
    quantize=(4 < 32),
    precision=[
        4,
        4,
        4+3
    ],
    learning_rate=0.0125,
)
    
# to set the map location
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model(torch.randn((16, 16)))  # Update tensor shapes 
model_param = torch.load(model_path, map_location=device)
model.load_state_dict(model_param['state_dict'])

metric = NeuralEfficiency(model, 
                          ne_dataloader, 
                          performance=0.732, 
                          max_batches=10000)

# store the result on file
if not os.path.exists(NE_DATA_DIR):
    os.mkdir(NE_DATA_DIR)

result = metric.compute(beta=beta)
metric.save_on_file(path=NE_DATA_DIR)
    
plot_neural_efficiency_per_layer(result['layers_efficiency'], 
                                 'Jet Tagger neural efficiency\n' \
                                f'{ranking_robust[best_index]}')

TypeError: must be real number, not NoneType